In [1]:
import rail
from rail.estimation.algos.deepdisc_test import *
from rail.core.data import TableHandle, JsonHandle, ModelHandle
from rail.core.stage import RailStage

In [2]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [3]:
model_file = "/home/g4merz/rail_deepdisc//test_informer.pkl"
model_handle = DS.add_data("model", data=None, handle_class=ModelHandle, path=model_file)
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

cfgfile = "./configs/solo/solo_swin_DC2.py"

In [4]:
deep_estimation_dict = dict(
    chunk_size=5,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    model=model_file,
    **deep_estimation_dict,
)

In [5]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Caching data
Processing chunk (start:end) - (0:5)
0
Matching objects - rank: 0
[01/17 16:50:00 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Matching objects - rank: 1
[01/17 16:50:00 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/17 16:50:00 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/17 16:50:00 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/17 16:50:00 d2.data.common]: Serialized dataset takes 1.29 MiB
[01/17 16:50:00 d2.data.common]: Serialized dataset takes 1.29 MiB


/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Returned pdfs - rank: 1
shapes (2,) (2, 200) (2,)
1 - calling gather_object
Returned pdfs - rank: 0
shapes (3,) (3, 200) (3,)
0 - making output list
Adding PDFs to ensemble
Adding all_pdfs to qp.Ensemble. rank - 0
Adding ensemble to the queue. rank - 0
Number of items in the queue: 1
Writing out this temporary ensemble to disk
Processing chunk (start:end) - (5:10)
0
Matching objects - rank: 0
[01/17 16:50:41 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Matching objects - rank: 1
[01/17 16:50:41 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/17 16:50:41 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/17 16:50:41 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/17 16:50:41 d2.data.common]: Serialized dataset takes 1.37 MiB
[01/17 16:50:41 d2.data.common]: Serialized dataset takes 1.37 MiB


/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Returned pdfs - rank: 1
shapes (4,) (4, 200) (4,)
1 - calling gather_object
Returned pdfs - rank: 0
shapes (5,) (5, 200) (5,)
0 - making output list
Adding PDFs to ensemble
Adding all_pdfs to qp.Ensemble. rank - 0
Adding ensemble to the queue. rank - 0
Number of items in the queue: 1
Writing out this temporary ensemble to disk
Inserting handle into data store.  output_DeepDiscEstimatorWithChunks: inprogress_output_DeepDiscEstimatorWithChunks.hdf5, DeepDiscEstimatorWithChunks


In [7]:
res_ens = results_from_chunks.read()
res_ens.npdf

9

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
x = np.linspace(0,3,100)
ax.plot(x, res_ens[0].pdf(x)[0])
ax.plot(x, res_ens[1].pdf(x)[0])